In [9]:
%reload_ext autoreload
%autoreload 2

In [6]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR, RAW_DATA_DIR

# load_dotenv(PARENT_DIR / '.env')
load_dotenv()

HOPSWORKS_API_KEY  = os.environ['HOPSWORKS_API_KEY']
HOPSWORKS_PROJECT_NAME = os.getenv('HOPSWORKS_PROJECT_NAME')

In [4]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

# from_year = 2022
# to_year = datetime.now().year
# print(f'Downloading raw data from {from_year} to {to_year}')

# rides = pd.DataFrame()
# for year in range(from_year, to_year+1):

#     rides_one_year = load_raw_data(year)

#     #append rows
#     rides = pd.concat([rides, rides_one_year])

In [5]:
rides = load_raw_data(2024, [4,5,6,7,8])

local file:  C:\Users\ingfz\ML Projects\taxi_demand_predictor\data\raw\rides_2024-04.parquet
File 2024-04 was already in local storage
local file:  C:\Users\ingfz\ML Projects\taxi_demand_predictor\data\raw\rides_2024-05.parquet
File 2024-05 was already in local storage
local file:  C:\Users\ingfz\ML Projects\taxi_demand_predictor\data\raw\rides_2024-06.parquet
local file:  C:\Users\ingfz\ML Projects\taxi_demand_predictor\data\raw\rides_2024-07.parquet
2024-07 file is not available
local file:  C:\Users\ingfz\ML Projects\taxi_demand_predictor\data\raw\rides_2024-08.parquet
2024-08 file is not available


In [7]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

100%|██████████| 261/261 [00:01<00:00, 197.43it/s]


In [12]:
ts_data

,pickup_hour,rides,pickup_location_id
0,2024-04-01 00:00:00,2,10
1,2024-04-01 01:00:00,1,10
2,2024-04-01 02:00:00,0,10
3,2024-04-01 03:00:00,0,10
4,2024-04-01 04:00:00,0,10
...,...,...,...
570019,2024-06-30 19:00:00,0,5
570020,2024-06-30 20:00:00,0,5
570021,2024-06-30 21:00:00,0,5
570022,2024-06-30 22:00:00,0,5


In [13]:
import hopsworks

In [14]:
project = hopsworks.login(
    project=HOPSWORK_PROJECT_NAME,
    api_key_value=HOPSWORK_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/424087


In [16]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [18]:
FEATURE_GROUP_NAME = 'time_series_hourly_feature_group'
FEATURE_GROUP_VERSION = 1

In [19]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description='Time-series data at hourly frequency',
    primary_key=['pickup_location_id','pickup_hour'],
    event_time='pickup_hour',
)

In [20]:
feature_group.insert(ts_data, write_options={'wait_for_job':False})

Uploading Dataframe: 0.00% |          | Rows 0/570024 | Elapsed Time: 00:00 | Remaining Time: ?

KafkaError: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}

In [29]:
query = feature_group.select_all().show(5)
print(query)

2024-08-04 21:58:15,236 ERROR: No commits found for featuregroup: /apps/hive/warehouse/forecast_model_featurestore.db/time_series_hourly_feature_group_1. Detail: Python exception: FlyingDuckException. gRPC client debug context: UNKNOWN:Error received from peer ipv4:3.19.160.248:5005 {created_time:"2024-08-05T01:58:15.0999852+00:00", grpc_status:2, grpc_message:"No commits found for featuregroup: /apps/hive/warehouse/forecast_model_featurestore.db/time_series_hourly_feature_group_1. Detail: Python exception: FlyingDuckException"}. Client context: IOError: Server never sent a data message. Detail: Internal
Traceback (most recent call last):
  File "c:\Users\ingfz\ML Projects\taxi_demand_predictor\taxienv\lib\site-packages\hsfs\core\arrow_flight_client.py", line 347, in afs_error_handler_wrapper
    return func(instance, *args, **kw)
  File "c:\Users\ingfz\ML Projects\taxi_demand_predictor\taxienv\lib\site-packages\hsfs\core\arrow_flight_client.py", line 405, in read_query
    return self

FeatureStoreException: No commits found for featuregroup: /apps/hive/warehouse/forecast_model_featurestore.db/time_series_hourly_feature_group_1. Detail: Python exception: FlyingDuckException. gRPC client debug context: UNKNOWN:Error received from peer ipv4:3.19.160.248:5005 {created_time:"2024-08-05T01:58:15.0999852+00:00", grpc_status:2, grpc_message:"No commits found for featuregroup: /apps/hive/warehouse/forecast_model_featurestore.db/time_series_hourly_feature_group_1. Detail: Python exception: FlyingDuckException"}. Client context: IOError: Server never sent a data message. Detail: Internal